* Firstly, let's add WaveBeans dependencies as well as logging library.
* Secondly, add all imports that might be useful.

In [1]:
%use lets-plot

@file:Repository("file:///home/jovyan/maven-local/repository")
@file:Repository("https://dl.bintray.com/wavebeans/wavebeans")
@file:DependsOn("io.wavebeans:lib:0.0.3-SNAPSHOT2")
@file:DependsOn("io.wavebeans:exe:0.0.3-SNAPSHOT2")
@file:DependsOn("io.wavebeans:http:0.0.3-SNAPSHOT2")
@file:DependsOn("ch.qos.logback:logback-classic:1.2.3")
@file:DependsOn("ch.qos.logback:logback-core:1.2.3")

import io.wavebeans.lib.*
import io.wavebeans.lib.io.*
import io.wavebeans.lib.math.*
import io.wavebeans.lib.stream.*
import io.wavebeans.lib.stream.fft.*
import io.wavebeans.lib.stream.window.*
import io.wavebeans.lib.table.*
import io.wavebeans.http.*
import io.wavebeans.execution.*
import java.util.concurrent.TimeUnit.*

* Start the HTTP Service so we could query all data in a different Jupyter notebook

In [2]:
val server = HttpService(serverPort=6800)
server.start()

io.wavebeans.http.HttpService@1c78a950

* Define our signal processing stream

In [3]:
val o = (880.sine() + 440.sine() + 220.sine())
    .trim(10000)
    .window(401)
    .fft(512)
    .toTable("fft", 10.s)

* Execute streams.

In [4]:
val overseer = LocalOverseer(listOf(o))
overseer.eval(44100.0f).all { it.get().finished }
overseer.close()